In [25]:

import signal
import threading
import cv2
import numpy as np
import json
from pyzbar import pyzbar
from PIL import Image

def get_qr_coords(camera_matrix, distortion_matrix, ret_qr, marker_size):
    return [_get_qr_coords(camera_matrix, distortion_matrix, qr, marker_size) for qr in ret_qr]

def _get_qr_coords(camera_matrix, distortion_matrix, ret_qr, marker_size):

    points = np.array(ret_qr.polygon)[None, :, :].astype(np.float32)
    
    #Selected coordinate points for each corner of QR code.
    edges = np.array([[0.,0.,0.],
                    [1.,0.,0.],
                    [1.,1.,0.],
                    [0.,1.,0.]])[:, None, :] * marker_size

    #determine the orientation of QR code coordinate system with respect to camera coorindate system.
    ret, rvec, tvec = cv2.solvePnP(edges, points, camera_matrix, distortion_matrix)

    #Define unit xyz axes. These are then projected to camera view using the rotation matrix and translation vector.
    unitv_points = np.array([[0.,0.,0.], [1.,0.,0.], [0.,1.,0.], [0.,0.,1.]])[:, None, :]
    if ret:
        points, _ = cv2.projectPoints(unitv_points, rvec, tvec, camera_matrix, distortion_matrix)
        rvec, _ = cv2.Rodrigues(rvec)
        return points, rvec, tvec
    return [], [], []

In [26]:


def read_camera_parameters(filepath = 'intrinsic.dat'):

    inf = open(filepath, 'r')

    cmtx = []
    dist = []

    #ignore first line
    line = inf.readline()
    for _ in range(3):
        line = inf.readline().split()
        line = [float(en) for en in line]
        cmtx.append(line)

    #ignore line that says "distortion"
    line = inf.readline()
    line = inf.readline().split()
    line = [float(en) for en in line]
    dist.append(line)

    #cmtx = camera matrix, dist = distortion parameters
    return np.array(cmtx), np.array(dist)
cmtx, dist = read_camera_parameters()

In [27]:

cap = cv2.VideoCapture('test.mp4') 
while True:
        ret, img = cap.read()
        if ret == False: break
        ret_qr = pyzbar.decode(img)
        if ret_qr:
            axis_points, rvec, tvec = get_qr_coords(cmtx, dist, ret_qr, 1.)[0]

            #BGR color format
            colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (0,0,0)]

            #check axes points are projected to camera view.
            if len(axis_points) > 0:
                axis_points = axis_points.reshape((4,2))

                origin = (int(axis_points[0][0]),int(axis_points[0][1]) )

                for p, c in zip(axis_points[1:], colors[:3]):
                    p = (int(p[0]), int(p[1]))

                    #Sometimes qr detector will make a mistake and projected point will overflow integer value. We skip these cases. 
                    if origin[0] > 5*img.shape[1] or origin[1] > 5*img.shape[1]:break
                    if p[0] > 5*img.shape[1] or p[1] > 5*img.shape[1]:break

                    cv2.line(img, origin, p, c, 5)

        cv2.imshow('frame', img)

        k = cv2.waitKey(20)
        if k == 27: break #27 is ESC key.
cap.release()
cv2.destroyAllWindows()